In [153]:
from collections import ChainMap
from functools   import reduce
from itertools   import chain
from typing      import Optional
import operator 
import re
from helpers import data

In [119]:
rules, my_ticket, tickets = data(16, parser=lambda s: s.split("\n"), sep="\n\n")

Create one `tickets` list. 

In [120]:
# Remove text and combine
tickets.pop(0)
my_ticket = [int(n) for n in my_ticket[1].split(",")] # Only keep values

# Convert to lists of ints 
tickets = [[int(n) for n in row.split(",")] for row in tickets]

Create one `rules` dictionary. 

In [121]:
rules[:3]

['departure location: 26-715 or 727-972',
 'departure station: 45-164 or 175-960',
 'departure platform: 43-247 or 270-972']

In [122]:
def parse_rule(rule: str) -> dict[str, list[tuple]]:
    """Parse rules into dictionary of bounds.""" 
    name, rest = rule.split(":")
    d = { name: [] }
    
    bounds = rest.strip().split(" or ")
    for bound in bounds: 
        minimum, maximum = bound.split("-")
        d[name].append((int(minimum), int(maximum)))
    
    return d 

assert parse_rule("departure location: 26-715 or 727-972") == {'departure location': [(26, 715), (727, 972)]}

In [123]:
# Bitwise or between dictionaries combines them: d1 | d2
rules = reduce(operator.or_, map(parse_rule, rules), {})

# Previous method 
# rules = dict(ChainMap(*[d for d in map(parse_rule, rules)]))

**Part 1:** Find the error rate, the sum of invalid values for any field. 

In [124]:
def within(val: int, *bounds: list[tuple[int]]) -> bool:
    """Return whether val is within any of (minimum, maximum) tuples given in bounds."""
    for minimum, maximum in bounds:
        if minimum <= val <= maximum: 
            return True 
    return False

assert within(972, (26, 715), (727, 972))
assert not within(3, (4, 100))

In [125]:
flatten = chain.from_iterable

errors = [field for ticket in tickets 
                for field in ticket 
                if not within(field, *flatten(rules.values()))]

In [126]:
sum(errors)

21081

**Part 1, Improved:** The main thing I need to clean up is converting a list of rule strings into a dictionary of name to ranges. My way of using `ChainMap` seems incomprehensible. Updated that above. 

**Part 2:** Discard the invalid tickets. Determine the order of the fields on each ticket, assuming the rest of the tickets are valid.  Multiply the six fields on my ticket that start with the word "departure".

My strategy is to find a set of potential fields for each index in all tickets. Then I'll just recursively search all possible orderings until I find a valid one. The algorithm will assign a field to all indices with only one possible field, and for tickets with more than one possible field, try all. 

In [127]:
tickets = [my_ticket, *tickets]
valid_tickets = [ticket for ticket in tickets 
                        if all(within(field, *flatten(rules.values())) for field in ticket)]

In [166]:
def get_possible_fields(vals: list[int]) -> set[str]:
    """Return set of fields whose bounds all vals are within."""
    return {field for field, bounds in rules.items()
                  if all(map(lambda v: within(v, *bounds), vals))}

In [168]:
# Get mapping of index to set of possible fields 
possible_fields = {i: get_possible_fields(
                            map(operator.itemgetter(i), valid_tickets))
                       for i in range(len(rules))}

In [179]:
def assign(possible_fields: dict[int, set[str]], assignment: list[Optional[str]]) -> Optional[list[str]]:
    """Given mapping of index to possible fields, find an assignment 
    that satisfies all indices, or return None.
    """
    for i, fields in possible_fields.items():
        if assignment[i] == None: 
            for field in fields: 
                updated_assignment = assignment.copy()
                updated_assignment[i] = field 
                
                new = possible_fields.copy()
                for k in new:
                    if field in new[k]: 
                        new[k] -= set(field)
                
                worked = assign(new, updated_assignment)
                if worked:
                    return worked 
            return None

assign(possible_fields, [None for _ in range(len(rules))])

KeyboardInterrupt: 